## Green Credit Allocation Using ACO
EC Project

In [ ]:
#Importing Libraries

import random
import numpy as np
import pandas as pd


In [ ]:
 #Read data from CSV file
data = pd.read_csv("data.csv")

In [ ]:
# Define risk categories and corresponding rating ranges
risk_categories = {
    "Safe": ["AAA"],
    "Adequately Safe": ["AA", "A"],
    "Moderate Risk": ["BBB", "BB"],
    "Risky": ["B", "CCC", "CC"],
    "Highly Risky": ["C"]
}


In [ ]:
# Define parameters
num_ants = 10
iterations = 15
pheromone_decay = 0.5
alpha = 1.0  # Weight for pheromone
beta = 1.5  # Weight for heuristic (adjust based on priorities)


In [ ]:
# Define normalization factors (find maximum values for each)
max_ytm = data['YTM'].max()
max_coupon = data['coupon'].max()
max_carbon_reduction = data['carbon_reduction'].max()


In [ ]:
def risk_factor(rating):
    for category, ratings in risk_categories.items():
        if rating in ratings:
            risk_factor_value = 5 - list(risk_categories.values()).index(ratings)
            return {"category": category, "factor": risk_factor_value}
    return {"category": "Highly Risky", "factor": 0}  # Default for unrated bonds


In [ ]:
def heuristic(i, investment, user_risk):
    # Combine YTM, coupon, carbon reduction, and risk factor (adjusted for user risk)
    ytm_norm = investment["YTM"] / max_ytm if not pd.isnull(investment["YTM"]) else 0
    coupon_norm = investment["coupon"] / max_coupon if not pd.isnull(investment["coupon"]) else 0

    # Check if "rating" column contains valid values
    if "rating" in investment and investment["rating"] in risk_categories:
        risk_factor_adj = risk_factor(investment["rating"])["factor"] * (1 - user_risk)
    else:
        risk_factor_adj = 0  # Assign default value if rating is missing or invalid

    return (ytm_norm + coupon_norm) * 0.02+ (investment["carbon_reduction"] / max_carbon_reduction) * 0.8 + risk_factor_adj * 0.22

In [ ]:
# ACO algorithm
def aco(data, user_risk):
    pheromone_matrix = np.ones((len(data), len(data)))
    ant_paths = []

    for _ in range(iterations):
        for _ in range(num_ants):
            current_city = random.randint(0, len(data) - 1)
            visited_cities = [current_city]
            while len(visited_cities) < len(data):
                # Calculate transition probabilities based on pheromone and heuristic
                probabilities = np.zeros(len(data))
                for i in range(len(data)):
                    if i not in visited_cities:
                        pheromone = pheromone_matrix[current_city, i]  # Pheromone on the edge
                        heuristic_value = heuristic(i, data.iloc[i], user_risk)  # Heuristic value of the next city
                        probabilities[i] = (pheromone**alpha) * (heuristic_value**beta)
                # Select next city based on probability distribution
                next_city = np.random.choice(np.arange(len(data)), p=probabilities / probabilities.sum())
                visited_cities.append(next_city)
                current_city = next_city

            # Update pheromone on the path (after completing the ant path)
            for i in range(1, len(visited_cities)):
                from_city, to_city = visited_cities[i-1], visited_cities[i]
                pheromone_matrix[from_city, to_city] += 1 / len(visited_cities)

            # Add the current ant path to the list of paths
            ant_paths.append(visited_cities)

        # Identify best path based on pheromone matrix (after all ant iterations)
        path_pheromone_sums = [sum(pheromone_matrix[path[i], path[i+1]] for i in range(len(path)-1)) for path in ant_paths]
        best_path_idx = np.argmax(path_pheromone_sums)

        # Retrieve best investment indices from best path
        best_investment_indices = ant_paths[best_path_idx]

    return best_investment_indices  # Return the best investment indices


In [ ]:
user_risk = 0.01
best_investments = aco(data, user_risk)

print("Recommended Investments:")
for i, investment_index in enumerate(best_investments[:10]): # Limiting to the first 10 investments
    if i >= 10:
        break
    investment = data.iloc[investment_index]
    print(f"  - ISIN: {investment['ISIN']} (Carbon Reduction: {investment['carbon_reduction']} Tonnes) (YTM: {investment['YTM']} )")


Recommended Investments:
  - ISIN: INE998Y07139 (Carbon Reduction: 20000 Tonnes) (YTM: 9.17 )
  - ISIN: INE053F07BD9 (Carbon Reduction: 12000 Tonnes) (YTM: 13.4 )
  - ISIN: INE040A08906 (Carbon Reduction: 15478 Tonnes) (YTM: 8.361 )
  - ISIN: INE140A07765 (Carbon Reduction: 80002 Tonnes) (YTM: 11.05 )
  - ISIN: INE146O08241 (Carbon Reduction: 60000 Tonnes) (YTM: 7.76 )
  - ISIN: INE756I07EP9 (Carbon Reduction: 74582 Tonnes) (YTM: 7.5825 )
  - ISIN: INE742F07528 (Carbon Reduction: 22154 Tonnes) (YTM: 8.4304 )
  - ISIN: INE476A08225 (Carbon Reduction: 95287 Tonnes) (YTM: 7.71 )
  - ISIN: INE134E07AT8 (Carbon Reduction: 45872 Tonnes) (YTM: 7.52 )
  - ISIN: INE774D08MW0 (Carbon Reduction: 45782 Tonnes) (YTM: 8.48 )


In [ ]:
# Call the ACO function and print the recommended investments
user_risk = 0.01
best_investments = aco(data, user_risk)

print("Recommended Investments:")
for i in best_investments:
    investment = data.iloc[i]
    print(f"  - ISIN: {investment['ISIN']} (Carbon Reduction: {investment['carbon_reduction']} Tonnes) (YTM: {investment['YTM']} )")


Recommended Investments:
  - ISIN: INE787H08055 (Carbon Reduction: 12485 Tonnes) (YTM: 8.49 )
  - ISIN: INE756I07EP9 (Carbon Reduction: 74582 Tonnes) (YTM: 7.5825 )
  - ISIN: INE721A08DA2 (Carbon Reduction: 87000 Tonnes) (YTM: 13.1 )
  - ISIN: INE134E08MT1 (Carbon Reduction: 12248 Tonnes) (YTM: 8.4 )
  - ISIN: INE140A07765 (Carbon Reduction: 80002 Tonnes) (YTM: 11.05 )
  - ISIN: INE476A08225 (Carbon Reduction: 95287 Tonnes) (YTM: 7.71 )
  - ISIN: INE124N07648 (Carbon Reduction: 19000 Tonnes) (YTM: 13.7951 )
  - ISIN: INE040A08666 (Carbon Reduction: 12458 Tonnes) (YTM: 9.31 )
  - ISIN: INE140R08080 (Carbon Reduction: 1425 Tonnes) (YTM: 8.1032 )
  - ISIN: INE084A08169 (Carbon Reduction: 10000 Tonnes) (YTM: 8.2318 )
  - ISIN: INE535H08801 (Carbon Reduction: 25874 Tonnes) (YTM: 8.3 )
  - ISIN: INE774D08MW0 (Carbon Reduction: 45782 Tonnes) (YTM: 8.48 )
  - ISIN: INE612U08041 (Carbon Reduction: 21587 Tonnes) (YTM: 9.18 )
  - ISIN: INE756I07EW5 (Carbon Reduction: 12485 Tonnes) (YTM: 8.55 )
  